## Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from pandas.tseries.offsets import DateOffset
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")


ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

## Data Loading And Preprocessing

Let's load the data and observe the first five lines.

In [ ]:
data = pd.read_csv("AirPassengers.csv")
data.head()

We convert the `Month` variable to date type.

In [ ]:
data["Month"] = pd.to_datetime(data["Month"])
data.head()

We assign the month variable as index. We edit variable and index names.

In [ ]:
data.set_index("Month", inplace=True)
data.columns = ["passengers"]
data.index.name = "date"
data.head()

Now we can look at the summary of the data set.

In [ ]:
data.info()
print(f"Dataset shape: {data.shape}")

## Train And Test Split

We reserve the last twelve months in the data for testing. We will train our model with the rest of the data.

In [ ]:
train_data = data[:len(data)-12]
test_data = data[len(data)-12:]

## Data Scaling

We scale the data between 0 and 1 for our LSTM model. We need to scale the data for all deep learning models.

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data)
train = scaler.transform(train_data)
test = scaler.transform(test_data)

## Create Model

We turn the data into a time series to train the LSTM model.

In [ ]:
n_input = 12
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)

Our model consists of a 200-node LSTM layer, a 0.15 Dropout layer and a one-node Dense layer as the output layer. We use Adam as the optimizer and mean squares error as the loss method.

In [ ]:
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit_generator(generator,epochs=90, verbose= 0)
model.summary()

After training our model, we prediction our test set and store it in an empty array.

In [ ]:
pred_list = []
batch = train[-n_input:].reshape((1, n_input, n_features))
for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

We turn our predictions into a data frame and combine it with our original data set.

In [ ]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),index=data[-n_input:].index, columns=['Prediction'])
df_test = pd.concat([data,df_predict], axis=1)

We have only prediction for the past twelve months. Let's view the last thirteen observations of the new data set.

In [ ]:
df_test.tail(13)

Let's visualize the results and observe the difference between the values ​​predicted by our model and the real values.

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df_test.index, df_test['passengers'])
plt.plot(df_test.index, df_test['Prediction'], color='r')
plt.legend(loc='best', fontsize='xx-large')
plt.xticks(fontsize=18, color= "white")
plt.yticks(fontsize=16, color= "white")
plt.show()

Here we predict twelve from the last twelve months of the training set.

## Future Forecast

Now, let's do the same things to predict the future. We will complete twelve months after the deadline in the data set.

In [ ]:
train = data
scaler.fit(train)
train = scaler.transform(train)
n_input = 12
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
model.fit_generator(generator,epochs=90, verbose= 0);

In [ ]:
pred_list = []  
batch = train[-n_input:].reshape((1, n_input, n_features))
for i in range(n_input):
    pred_list.append(model.predict(batch)[0])      
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

We add the next twelve months to the data set.

In [ ]:
add_dates = [data.index[-1] + DateOffset(months=x) for x in range(0,13) ]
future_dates = pd.DataFrame(index=add_dates[1:],columns=data.columns)
future_dates.head(12)

In [ ]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['Prediction'])

df_proj = pd.concat([data,df_predict], axis=1)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df_proj.index, df_proj['passengers'])
plt.plot(df_proj.index, df_proj['Prediction'], color='r')
plt.legend(loc='best', fontsize='xx-large')
plt.xticks(fontsize=18, color = "white")
plt.yticks(fontsize=16, color = "white")
plt.show()

Lost chart of our latest model.

In [ ]:
losses_lstm = model.history.history['loss']
plt.figure(figsize=(12,4))
plt.xlabel("Epochs", color = "white")
plt.ylabel("Loss", color = "white")
plt.xticks(  color = "white")
plt.yticks(  color = "white")
plt.plot(range(len(losses_lstm)),losses_lstm);